# Displaying Tables and Catalogs

This notebook shows how to display tables in Firefly and how to modify the table display (filtering/sorting). Catalog tables are overlaid on images by default if Firefly recognizes celestial coordinates in the table.

## Setup
First, we create a `FireflyClient` instance and open a Firefly viewer. See [Initializing a FireflyClient instance](./initializing-vanilla.html) for more details. Then, we display a FITS image from a URL so that we have an image for catalog overlays. See [Displaying a FITS Image](./displaying-fits-image.html) for more details.

In [1]:
from firefly_client import FireflyClient

# Following imports are just for example data fetching and processing (not needed always)
from astropy.utils.data import download_file
from astropy.table import Table
import io

# Initialize a FireflyClient instance
fc = FireflyClient.make_client(url="https://irsa.ipac.caltech.edu/irsaviewer")

# Display a FITS image (from URL of WISE W2 cutout of M51 galaxy)
cutout_image_url = 'https://irsa.ipac.caltech.edu/ibe/data/wise/allsky/4band_p1bm_frm/9a/05379a/141/05379a141-w2-int-1b.fits?center=202.4841667,47.23055556&size=400pix'
plot_id = 'wise-cutout'
fc.show_fits_image(file_input=cutout_image_url, plot_id=plot_id, title='WISE Cutout')

{'success': True}

## Display a Table

Use `show_table` method of the FireflyClient object to display a table. Similar to `show_fits_image`, it can take a URL, a local file path, or a file-like object as input via the `file_input` parameter.

### From a local file
You can specify path of any local table file. Here we download the Gaia DR3 catalog for 300" cone around M51 and use its file path:

In [2]:
tbl_url_gaia = "https://gea.esac.esa.int/tap-server/tap/sync?LANG=ADQL&QUERY=SELECT+*+FROM+gaiadr3.gaia_source+WHERE+CONTAINS(POINT('ICRS',ra,dec),CIRCLE('ICRS',202.4841667,47.23055556,0.083333))=1"
tbl_fpath_gaia = download_file(tbl_url_gaia, cache=True, timeout=120)
tbl_fpath_gaia

'/Users/jsinghal/.astropy/cache/download/url/55fa7a5603170b3f0b7f826748efe73a/contents'

In [3]:
tbl_id_gaia = 'gaia-catalog'
fc.show_table(file_input=tbl_fpath_gaia,
              tbl_id=tbl_id_gaia, 
              title='Gaia Catalog')

{'success': True}

### From an in-memory file-like object

You can also use a file-like object (e.g. `StringIO` or `BytesIO` stream) instead of a local file path or URL. This is useful when you are working with a Table file in memory (for e.g. as an Astropy Table) and don't want to write it to disk. 

In [4]:
gaia_tbl = Table.read(tbl_fpath_gaia)
gaia_galaxy_tbl = gaia_tbl[gaia_tbl['classprob_dsc_combmod_galaxy'] >= 0.95]
gaia_galaxy_tbl[:5]

solution_id,DESIGNATION,SOURCE_ID,random_index,ref_epoch,ra,ra_error,dec,dec_error,parallax,parallax_error,parallax_over_error,pm,pmra,pmra_error,pmdec,pmdec_error,ra_dec_corr,ra_parallax_corr,ra_pmra_corr,ra_pmdec_corr,dec_parallax_corr,dec_pmra_corr,dec_pmdec_corr,parallax_pmra_corr,parallax_pmdec_corr,pmra_pmdec_corr,astrometric_n_obs_al,astrometric_n_obs_ac,astrometric_n_good_obs_al,astrometric_n_bad_obs_al,astrometric_gof_al,astrometric_chi2_al,astrometric_excess_noise,astrometric_excess_noise_sig,astrometric_params_solved,astrometric_primary_flag,nu_eff_used_in_astrometry,pseudocolour,pseudocolour_error,ra_pseudocolour_corr,dec_pseudocolour_corr,parallax_pseudocolour_corr,pmra_pseudocolour_corr,pmdec_pseudocolour_corr,astrometric_matched_transits,visibility_periods_used,astrometric_sigma5d_max,matched_transits,new_matched_transits,matched_transits_removed,ipd_gof_harmonic_amplitude,ipd_gof_harmonic_phase,ipd_frac_multi_peak,ipd_frac_odd_win,ruwe,scan_direction_strength_k1,scan_direction_strength_k2,scan_direction_strength_k3,scan_direction_strength_k4,scan_direction_mean_k1,scan_direction_mean_k2,scan_direction_mean_k3,scan_direction_mean_k4,duplicated_source,phot_g_n_obs,phot_g_mean_flux,phot_g_mean_flux_error,phot_g_mean_flux_over_error,phot_g_mean_mag,phot_bp_n_obs,phot_bp_mean_flux,phot_bp_mean_flux_error,phot_bp_mean_flux_over_error,phot_bp_mean_mag,phot_rp_n_obs,phot_rp_mean_flux,phot_rp_mean_flux_error,phot_rp_mean_flux_over_error,phot_rp_mean_mag,phot_bp_rp_excess_factor,phot_bp_n_contaminated_transits,phot_bp_n_blended_transits,phot_rp_n_contaminated_transits,phot_rp_n_blended_transits,phot_proc_mode,bp_rp,bp_g,g_rp,radial_velocity,radial_velocity_error,rv_method_used,rv_nb_transits,rv_nb_deblended_transits,rv_visibility_periods_used,rv_expected_sig_to_noise,rv_renormalised_gof,rv_chisq_pvalue,rv_time_duration,rv_amplitude_robust,rv_template_teff,rv_template_logg,rv_template_fe_h,rv_atm_param_origin,vbroad,vbroad_error,vbroad_nb_transits,grvs_mag,grvs_mag_error,grvs_mag_nb_transits,rvs_spec_sig_to_noise,phot_variable_flag,l,b,ecl_lon,ecl_lat,in_qso_candidates,in_galaxy_candidates,non_single_star,has_xp_continuous,has_xp_sampled,has_rvs,has_epoch_photometry,has_epoch_rv,has_mcmc_gspphot,has_mcmc_msc,in_andromeda_survey,classprob_dsc_combmod_quasar,classprob_dsc_combmod_galaxy,classprob_dsc_combmod_star,teff_gspphot,teff_gspphot_lower,teff_gspphot_upper,logg_gspphot,logg_gspphot_lower,logg_gspphot_upper,mh_gspphot,mh_gspphot_lower,mh_gspphot_upper,distance_gspphot,distance_gspphot_lower,distance_gspphot_upper,azero_gspphot,azero_gspphot_lower,azero_gspphot_upper,ag_gspphot,ag_gspphot_lower,ag_gspphot_upper,ebpminrp_gspphot,ebpminrp_gspphot_lower,ebpminrp_gspphot_upper,libname_gspphot
,,,,yr,deg,mas,deg,mas,mas,mas,,mas / yr,mas / yr,mas / yr,mas / yr,mas / yr,,,,,,,,,,,,,,,,,mas,,,,1 / um,1 / um,1 / um,,,,,,,,mas,,,,,deg,,,,,,,,deg,deg,deg,deg,,,electron / s,electron / s,,mag,,electron / s,electron / s,,mag,,electron / s,electron / s,,mag,,,,,,,mag,mag,mag,km / s,km / s,,,,,,,,d,km / s,K,log(cm.s**-2),dex,,km / s,km / s,,mag,mag,,,,deg,deg,deg,deg,,,,,,,,,,,,,,,K,K,K,log(cm.s**-2),log(cm.s**-2),log(cm.s**-2),dex,dex,dex,pc,pc,pc,mag,mag,mag,mag,mag,mag,mag,mag,mag,
int64,object,int64,int64,float64,float64,float32,float64,float32,float64,float32,float32,float32,float64,float32,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,int16,int16,int16,float32,float32,float32,float32,int16,bool,float32,float32,float32,float32,float32,float32,float32,float32,int16,int16,float32,int16,int16,int16,float32,float32,int16,int16,float32,float32,float32,float32,float32,float32,float32,float32,float32,bool,int16,float64,float32,float32,float32,int16,float64,float32,float32,float32,int16,float64,float32,float32,float32,float32,int16,int16,int16,int16,int16,float32,float32,float32,float32,float32,int16,int16,int16,int16,float32,float32,float32,float32,float32,float32,float32,float32,

In [5]:
tbl_stream = io.BytesIO()
gaia_galaxy_tbl.write(tbl_stream, format='votable')

In [6]:
fc.show_table(file_input=tbl_stream,
              tbl_id=tbl_id_gaia+'-galaxy',
              title='Gaia Galaxy Catalog')

{'success': True}

### From a URL

You can specify URL of any table. Here we use an IRSA TAP query to retrieve a 2MASS catalog for 0.125 degree cone at M51.

In [7]:
tbl_url_2mass = "http://irsa.ipac.caltech.edu/TAP/sync?FORMAT=IPAC_TABLE&QUERY=SELECT+*+FROM+fp_psc+WHERE+CONTAINS(POINT('J2000',ra,dec),CIRCLE('J2000',202.4841667,47.23055556,0.125))=1"

In [8]:
tbl_id_2mass = '2mass-catalog'
fc.show_table(file_input=tbl_url_2mass,
              tbl_id=tbl_id_2mass,
              title='2MASS Catalog')

{'success': True}

.. note::
    Displaying a catalog table also caused Firefly to overlay the image with markers for each row of the table by default. Also we see the active chart and coverage tab as bonus! If we have more than one images displayed, the catalog overlay will appear to all images that cover the catalog area.

## Modify the displayed table
After displaying a table in Firefly, you can apply filters and sorting by `tbl_id`.

### Sorting by a column
You can sort the table by a column in ascending (`ASC`) or descending (`DESC`) order:

In [9]:
fc.sort_table_column(tbl_id=tbl_id_2mass,  # note: we use the table id we defined above
                     column_name='j_m', sort_direction='ASC')

{'success': True}

### Filtering the rows
Filters can be specified as an SQL WHERE clause-like string with column names quoted:

In [10]:
fc.apply_table_filters(tbl_id=tbl_id_2mass,
                       filters='"j_m"<15 and "j_cmsig"<0.05')

{'success': True}

.. note::
    You can see these filters in the table display. The catalog markers on the image also reduce in number as well as the active chart updates too.

To remove the filters:

In [11]:
fc.apply_table_filters(tbl_id=tbl_id_2mass, filters='')

{'success': True}

## Tweaking the table display parameters
`show_table` method offers several parameters other than the ones we used above, to control how to display the table.

### Load table without displaying it
By default, `show_table` loads the provided table and displays it in a table viewer. If it is desired to overlay the table on an image, or to make plots from it without showing the table in the viewer, you can set the `visible` parameter to False. 

.. tip::
    This is especially useful for loading a MOC file (a single-column table) since you only want to overlay the MOC map on the HiPS image without displaying the table. See [Overlay MOC on HiPS](./displaying-hips.html)

In [12]:
tbl_url_spitzer = "https://irsa.ipac.caltech.edu/cgi-bin/Gator/nph-query?outfmt=1&catalog=slphotdr4&objstr=202.484167d+47.230556d+eq+j2000&spatial=Cone&radius=240"
fc.show_table(file_input=tbl_url_spitzer,
              title='Spitzer catalog',
              visible=False)

{'success': True}

.. note::
    It may take some time to load this catalog table, and since there's no feedback from the table display, look for the spinner on the top left of the image display.

### Display table without catalog overlay

If the table contains celestial coordinates recognized by Firefly, the catalog overlay will appear by default on the displayed image. But if you specifically do not want the table overlaid on the image, you can set the `is_catalog` parameter to False

In [13]:
fc.show_table(file_input=tbl_url_spitzer,
              title='Spitzer: no overlay',
              is_catalog=False
              )

{'success': True}